In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.preprocessing import OneHotEncoder

In [5]:
df = pd.read_csv(r"../ML_Challenge/train.csv")

In [6]:
df.head()
no_rows = df.shape[0]
no_columns = df.shape[1]

In [7]:
df = df.drop(['customer_id','Name','security_no','referral_id'],axis = 1)

In [8]:
df.head()

,age,gender,region_category,membership_category,joining_date,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,...,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,18,F,Village,Platinum Membership,2017-08-17,No,Gift Vouchers/Coupons,?,Wi-Fi,16:08:02,...,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,2
1,32,F,City,Premium Membership,2017-08-28,?,Gift Vouchers/Coupons,Desktop,Mobile_Data,12:38:13,...,306.34,12838.38,10.0,NaN,Yes,No,Yes,Solved,Quality Customer Care,1
2,44,F,Town,No Membership,2016-11-11,Yes,Gift Vouchers/Coupons,Desktop,Wi-Fi,22:53:21,...,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,5
3,37,M,City,No Membership,2016-10-29,Yes,Gift Vouchers/Coupons,Desktop,Mobile_Data,15:57:50,...,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,5
4,31,F,City,No Membership,2017-09-12,No,Credit/Debit Card Offers,Smartphone,Mobile_Data,15:46:44,...,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,5


In [9]:
df['last_visit_time']

0        16:08:02
1        12:38:13
2        22:53:21
3        15:57:50
4        15:46:44
           ...   
36987    04:14:05
36988    23:18:31
36989    03:50:25
36990    09:50:03
36991    01:39:52
Name: last_visit_time, Length: 36992, dtype: object

In [10]:
 #This step converts all the missing and irrelevant data  present in the dataframe to NaN
def convert_to_NaN(df):
    for column in df:
        df[column] = df[column].apply(lambda x: np.nan if x == '?' or x == 'Error' else x)        
    df.fillna(0,inplace = True)
    return df    

In [11]:
df = convert_to_NaN(df)

In [12]:
def convert_time(df):
    df['last_visit_time'] = pd.to_datetime(df['last_visit_time'],format = '%H:%M:%S')
    reference_time = ['23:59:59']*no_rows
    df['reference_time'] = reference_time
    df['reference_time'] = pd.to_datetime(df['reference_time'],format = '%H:%M:%S')
    df['seconds_since_last_visit'] = ( df['reference_time'] - df['last_visit_time'] ).dt.total_seconds()
    #df['seconds_since_last_visit']
    df = df.drop(['last_visit_time','reference_time'],axis = 1)
    return df

In [13]:
def convert_date(df):
    df['joining_date'] = pd.to_datetime(df['joining_date'], format = "%Y-%m-%d")
    const = ['2021-12-31']*no_rows
    df['const'] = const
    df['const'] = pd.to_datetime(df['const'], format = "%Y-%m-%d")
    #df['const']
    df['days_since_joining'] = df['const'] - df['joining_date']    
    df['days_since_joining'] = df['days_since_joining'] / np.timedelta64(1,'D')
    df = df.drop(['joining_date','const'],axis = 1)
    #df
    return df

In [14]:
def Encoder(df):
          columnsToEncode = list(df.select_dtypes(include=['category','object']))
          le = LabelEncoder()
          for feature in columnsToEncode:
              df[feature] = le.fit_transform(df[feature].astype(str))  
              try:
                  df[feature] = le.fit_transform(df[feature])
              except:
                  print('Error encoding '+feature)
          return df

In [15]:
df = convert_time(df)
df = convert_date(df)
df = Encoder(df)
df

,age,gender,region_category,membership_category,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,days_since_last_login,avg_time_spent,...,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score,seconds_since_last_visit,days_since_joining
0,18,0,3,3,1,2,0,2,17,300.630000,...,747,781.750000,1,1,0,1,4,2,28317.0,1597.0
1,32,0,1,4,0,2,2,1,16,306.340000,...,733,0.000000,1,0,1,2,5,1,40906.0,1586.0
2,44,0,2,2,2,2,2,2,14,516.160000,...,780,500.690000,0,1,1,3,3,5,3998.0,1876.0
3,37,1,1,2,2,2,2,1,11,53.270000,...,1638,567.660000,0,1,1,4,3,5,28929.0,1889.0
4,31,0,1,2,1,1,3,1,20,113.130000,...,744,663.060000,0,1,1,2,3,5,29595.0,1571.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36987,46,0,0,0,1,1,2,2,2,-650.682759,...,1638,639.510000,0,1,1,0,0,4,71154.0,1562.0
36988,29,0,2,0,1,3,3,2,13,-638.123421,...,810,527.990000,1,0,0,1,1,5,2488.0,2013.0
36989,23,0,0,0,2,2,2,2,12,154.940000,...,683,680.470000,0,1,1,4,3,4,72574.0,1937.0
36990,53,1,3,3,1,2,3,1,15,482.610000,...,775,197.264414,1,1,0,1,0,3,50996.0,1660.0


In [16]:
from sklearn.preprocessing import MinMaxScaler
Scaler = MinMaxScaler()
df[['avg_time_spent','avg_transaction_value','avg_frequency_login_days','days_since_last_login','points_in_wallet','seconds_since_last_visit','days_since_joining']] = Scaler.fit_transform(df[['avg_time_spent','avg_transaction_value','avg_frequency_login_days','days_since_last_login','points_in_wallet','seconds_since_last_visit','days_since_joining']])
df

,age,gender,region_category,membership_category,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,days_since_last_login,avg_time_spent,...,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score,seconds_since_last_visit,days_since_joining
0,18,0,3,3,1,2,0,2,0.991220,0.514859,...,0.451906,0.545073,1,1,0,1,4,2,0.327747,0.124201
1,32,0,1,4,0,2,2,1,0.990244,0.515803,...,0.443436,0.268810,1,0,1,2,5,1,0.473455,0.114155
2,44,0,2,2,2,2,2,2,0.988293,0.550486,...,0.471869,0.445750,0,1,1,3,3,5,0.046274,0.378995
3,37,1,1,2,2,2,2,1,0.985366,0.473971,...,0.990926,0.469416,0,1,1,4,3,5,0.334830,0.390868
4,31,0,1,2,1,1,3,1,0.994146,0.483866,...,0.450091,0.503130,0,1,1,2,3,5,0.342539,0.100457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36987,46,0,0,0,1,1,2,2,0.976585,0.357610,...,0.990926,0.494807,0,1,1,0,0,4,0.823551,0.092237
36988,29,0,2,0,1,3,3,2,0.987317,0.359686,...,0.490018,0.455397,1,0,0,1,1,5,0.028797,0.504110
36989,23,0,0,0,2,2,2,2,0.986341,0.490777,...,0.413188,0.509282,0,1,1,4,3,4,0.839987,0.434703
36990,53,1,3,3,1,2,3,1,0.989268,0.544940,...,0.468845,0.338522,1,1,0,1,0,3,0.590238,0.181735


In [17]:
Y = df.iloc[:,-3].values
Y
df = df.drop(['churn_risk_score'],axis = 1)
df

,age,gender,region_category,membership_category,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,seconds_since_last_visit,days_since_joining
0,18,0,3,3,1,2,0,2,0.991220,0.514859,0.526717,0.451906,0.545073,1,1,0,1,4,0.327747,0.124201
1,32,0,1,4,0,2,2,1,0.990244,0.515803,0.121456,0.443436,0.268810,1,0,1,2,5,0.473455,0.114155
2,44,0,2,2,2,2,2,2,0.988293,0.550486,0.204074,0.471869,0.445750,0,1,1,3,3,0.046274,0.378995
3,37,1,1,2,2,2,2,1,0.985366,0.473971,0.246577,0.990926,0.469416,0,1,1,4,3,0.334830,0.390868
4,31,0,1,2,1,1,3,1,0.994146,0.483866,0.238950,0.450091,0.503130,0,1,1,2,3,0.342539,0.100457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36987,46,0,0,0,1,1,2,2,0.976585,0.357610,0.267140,0.990926,0.494807,0,1,1,0,0,0.823551,0.092237
36988,29,0,2,0,1,3,3,2,0.987317,0.359686,0.103611,0.490018,0.455397,1,0,0,1,1,0.028797,0.504110
36989,23,0,0,0,2,2,2,2,0.986341,0.490777,0.376609,0.413188,0.509282,0,1,1,4,3,0.839987,0.434703
36990,53,1,3,3,1,2,3,1,0.989268,0.544940,0.015925,0.468845,0.338522,1,1,0,1,0,0.590238,0.181735


In [18]:
#used here to transform categorical data
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('cat', OneHotEncoder(), ['gender','region_category','membership_category','joined_through_referral','preferred_offer_types','medium_of_operation','internet_option','used_special_discount','offer_application_preference','past_complaint','complaint_status','feedback'])], remainder='passthrough')
X = np.array(ct.fit_transform(df))

In [20]:
X

array([[1.        , 0.        , 0.        , ..., 0.54507345, 0.32774685,
        0.12420091],
       [1.        , 0.        , 0.        , ..., 0.26881044, 0.47345455,
        0.11415525],
       [1.        , 0.        , 0.        , ..., 0.44574952, 0.04627368,
        0.37899543],
       ...,
       [1.        , 0.        , 0.        , ..., 0.50928206, 0.83998657,
        0.4347032 ],
       [0.        , 1.        , 0.        , ..., 0.33852181, 0.59023831,
        0.18173516],
       [0.        , 1.        , 0.        , ..., 0.52324099, 0.93064735,
        0.73059361]])

In [19]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [23]:
X_train.shape

(27744, 55)

In [24]:
X_test.shape

(9248, 55)

In [25]:
def dist(x1,x2):
    return np.sqrt(sum((x1-x2)**2))

def knn(X,Y,queryPoint,k=5):
    
    vals = []
    m = X.shape[0]
    
    for i in range(m):
        d = dist(queryPoint,X[i])
        vals.append((d,Y[i]))
        
    
    vals = sorted(vals)
    # Nearest/First K points
    vals = vals[:k]
    
    vals = np.array(vals)
    
    #print(vals)
    
    new_vals = np.unique(vals[:,1],return_counts=True)
    #print(new_vals)
    
    index = new_vals[1].argmax()
    pred = new_vals[0][index]
    
    return pred

In [35]:
pred = knn(X,Y,X_test[96])

print(int(pred))

3


In [36]:
y_test[96]

5

In [ ]:
Y_pred=[]
for i in range(X.shape[0]):
    Y_pred.append(knn(X,Y,X_test[i]))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, Y)